In [ ]:
import lyricsgenius
import pandas as pd
import datetime as dt
import time
import os

import certifi
certifi.where() #incase you need to add ssl certs

In [ ]:
genius = lyricsgenius.Genius("rl0rzlsvRISoZV_yKGB7fzmxoo7RxC-wEG_uuZtQVAIpWtdZB6fAPGbY_Zv7K5pg")
genius.verbose = False # Turn off status messages
genius.remove_section_headers = True # Remove section headers (e.g. [Chorus]) from lyrics when searching
genius.skip_non_songs = False # Include hits thought to be non-songs (e.g. track lists)
genius.excluded_terms = ["(Remix)", "(Live)"] # Exclude songs with these words in their title

# Test

In [ ]:
artist = genius.search_artist("Andy Shauf", max_songs=3, sort="title")
song = genius.search_song("To You", artist.name)
print(artist.songs)
print(song.lyrics)

# all Christmas top 40s

In [ ]:
def populate_lyrics(df, savename):

    count = 0
    df["LYRICS"] = None
    for index, row in df.iterrows():
        
        print(f"{dt.datetime.now().strftime('%H:%M %d/%m/%Y')} - Searching for {row['Artist']} and  {row['Song']}", end = " : ")
        
        # search for the artist
        try:
            artist = genius.search_artist(row["Artist"], sort="popularity", max_songs=3)
        except:
            continue
        if artist is None:
            print("Artist not available")
            continue
        
        # search for the song lyrics
        try:
            song = genius.search_song(row["Song"], artist.name)
        except:
            continue
        if song is None:
            print("Song not available")
            continue
        
        # add to the dataframe
        df.loc[index,"LYRICS"] = song.lyrics
        
        #
        print(f"Lyrics added")
        
        # dont overload the website
        time.sleep(1)
        
        count+=1
        
        # save regularly
        if count%20 == 0:
            save_df(df, savename)
        
    save_df(df, savename)
        
    return df

def save_df(df, name):
    print(f"{dt.datetime.now().strftime('%H:%M %d/%m/%Y')} - SAVING")
    df.replace("\n", "--").to_csv(f"{name}.csv", )
    df.reset_index().to_feather(f"{name}.feather")
    df.to_pickle(f"{name}.pkl")

In [ ]:
start = dt.datetime.now().strftime("%H:%M %d/%m/%Y")
print(start)

chart_file_path = r"C:\Users\rachel.hassall\OneDrive - National Grid\GitHub\NG-DSc-Christmas-Jamboree\data\Weekly top 100 2010-2020 (Singles)\singles_2009_2020.csv"


chart_file_path = r"C:\Users\rachel.hassall\OneDrive - National Grid\GitHub\NG-DSc-Christmas-Jamboree\data\Weekly top 100 2010-2020 (Singles)\singles_2009_2020_lyrics Part 3.csv"
savename = r"C:\Users\rachel.hassall\OneDrive - National Grid\GitHub\NG-DSc-Christmas-Jamboree\data\Weekly top 100 2010-2020 (Singles)\singles_2009_2020_lyrics Part 3"

df = pd.read_csv(chart_file_path)

# get list of unique artist/song combos
df = df.groupby(["Song", "Artist"]).nunique().reset_index()[["Artist", "Song"]]

print(f"{len(df)} rows to get lyrics for")

df = populate_lyrics(df, savename)
    

In [ ]:
df.tail()

# all christmas number 1s

In [ ]:
#df = pd.read_csv(r"C:\Users\rachel.hassall\Downloads\All_xmas_no1s.csv")
df = df.drop("Unnamed: 0", axis=1)
df.head()

In [ ]:
df["LYRICS"] = None
for index, row in df.iterrows(): 
    print(row['ARTIST'], row['TITLE'])
    
    # search for the artist
    try:
        artist = genius.search_artist(row["ARTIST"], sort="popularity", max_songs=3)
    except:
        continue
    if artist is None:
        print("Artist not available")
        continue
    
    # search for the song lyrics
    try:
        song = genius.search_song(row["TITLE"], artist.name)
    except:
        continue
    if song is None:
        print("Song not available")
        continue
    
    # add to the dataframe
    df.loc[index,"LYRICS"] = song.lyrics
    
    # dont overload the website
    time.sleep(1)

In [ ]:
# check the data
df.head()

In [ ]:
# remove the weird stuff
df.loc[13,"LYRICS"] = None

In [ ]:
df.replace("\n", "--").to_csv("All_xmas_no1s_with_lyrics.csv", )

In [ ]:
df.reset_index().to_feather("All_xmas_no1s_with_lyrics.feather")

In [ ]:
df

In [ ]:
##get list of files
#file_dict = {}
#
#for root, directories, files in os.walk(r"C:\Users\rachel.hassall\OneDrive - National Grid\GitHub\NG-DSc-Christmas-Jamboree\data\Christmas Top 40s",):
#    for file in files:
#        name = os.path.splitext(file)[0]
#        filetype = os.path.splitext(file)[1]
#        path = os.path.join(root, file)
#        
#        if filetype==".csv":
#            file_dict[name] = path

In [ ]:
#df_dict = {}
#
## iterate from 2019 backwards in case takes too long
#for name in sorted(file_dict.keys(), reverse=True):
#    path = file_dict[name]
#    df = pd.read_csv(path)
#    try:
#        print("-------------------------------------------------------------------------------------------------------------------")
#        print(name)
#        df = df.drop("Unnamed: 0", axis=1)
#        df = df.rename({"Song" :"TITLE", "Artist" : "ARTIST"}, axis=1)
#        df = populate_lyrics(df)
#        df_dict[name]=df
#        print(f"{name} COMPLETE")
#    except Exception as E:
#        print(f"{name} failed")
#        print(E)